In [2]:
import sys
sys.path.append('../..')

import eeg_decoder 
import pandas as pd
import numpy as np
import scipy.io as io
from pathlib import Path
import shutil

In [5]:
experiment = 'e'
data_dir = f'./data/{experiment}'
exp = eeg_decoder.Experiment(experiment, data_dir, dev=False,info_from_file=False)
sub_folders = [f for f in Path(exp.data_dir).iterdir() if f.is_dir()]
sub_folders

[WindowsPath('data/e/01'),
 WindowsPath('data/e/02'),
 WindowsPath('data/e/04'),
 WindowsPath('data/e/05'),
 WindowsPath('data/e/06'),
 WindowsPath('data/e/07'),
 WindowsPath('data/e/08'),
 WindowsPath('data/e/10'),
 WindowsPath('data/e/11'),
 WindowsPath('data/e/13'),
 WindowsPath('data/e/14'),
 WindowsPath('data/e/15'),
 WindowsPath('data/e/16'),
 WindowsPath('data/e/18'),
 WindowsPath('data/e/19')]

In [4]:
for isub, sub_folder in enumerate(sub_folders):
    print(isub)
    exp.xdata_files = list(sub_folder.rglob('*xdata.mat'))
    exp.ydata_files = list(sub_folder.rglob('*ydata.mat'))
    exp.behavior_files = list(sub_folder.rglob('*behavior.csv'))
    exp.artifact_idx_files = list(sub_folder.rglob('*artifact_idx.mat'))        

    # load and concatenate data from behavior, eeg, and trial labels
    for irun in range(len(exp.xdata_files)): 
        x1, y1 = exp.load_eeg(irun)
        idx1 = exp.load_artifact_idx(irun)
        beh1 = exp.load_behavior(irun, remove_artifact_trials=False)
        
        output_check = f'Beh=Idx: {len(pd.DataFrame(beh1)) == len(idx1)}, X=Idx: {len(x1) == sum(idx1)}'
        print(output_check)
        
        if irun == 0:
            x = x1 
            y = y1
            idx = idx1
            beh = pd.DataFrame(beh1)
            sub_num = sub_folder.name
        else:
            x = np.concatenate((x, x1), 0)
            y = np.concatenate((y, y1), 0)
            idx = np.concatenate((idx, idx1), 0)
            beh1_df = pd.DataFrame(beh1)
            beh = pd.concat((beh, beh1_df))

    x = np.moveaxis(x, 0, 2) # roll bc saving with savemat changes shape for some reason...
    print(np.unique(y, return_counts=True))

    # save
    io.savemat(f'data/{experiment}/{experiment}_{sub_num}_xdata.mat',{'xdata':x})
    io.savemat(f'data/{experiment}/{experiment}_{sub_num}_ydata.mat',{'ydata':y})
    io.savemat(f'data/{experiment}/{experiment}_{sub_num}_artifact_idx.mat',{'artifact_idx':idx})
    beh.to_csv(f'data/{experiment}/{experiment}_{sub_num}_behavior.csv')
    shutil.copy(f'data/{experiment}/{sub_num}/{experiment}_{sub_num}_1_info.mat',f'data/{experiment}/{experiment}_{sub_num}_info.mat')

0
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
(array([10, 11, 12, 20, 21, 22], dtype=uint8), array([ 39, 207, 216,  44, 226, 216], dtype=int64))
1
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
(array([10, 11, 12, 20, 21, 22], dtype=uint8), array([ 47, 254, 252,  47, 208, 200], dtype=int64))
2
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
(array([10, 11, 12, 20, 21, 22], dtype=uint8), array([ 42, 212, 199,  40, 216, 208], dtype=int64))
3
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True
(array([10, 11, 12, 20, 21, 22], dtype=uint8), array([ 37, 182, 202,  43, 168, 206], dtype=int64))
4
Beh=Idx: True, X=Idx: True
Beh=Idx: True, X=Idx: True


FileNotFoundError: [Errno 2] No such file or directory: 'data/e/bad/e_bad_1_info.mat'